In [14]:
import pandas as pd
df = pd.read_csv("/users/wiley/Documents/Downloads/all_matches.csv")
for i in range(0, 10):
    batch_df = pd.read_csv(f"/users/wiley/Documents/Downloads/all_matches_batch_{i}.csv")
    df = pd.concat([df, batch_df], ignore_index=True)
df.to_csv("/users/wiley/Documents/Downloads/all_matches_final.csv", index=False)

In [15]:
len(df)

64130

In [16]:
df.head()

,Player,Role,Level,Kills,Deaths,Assists,KDA,CS,CS in Team's Jungle,CS in Enemy Jungle,...,Total Time Spent Dead,Consumables purchased,Items Purchased,Shutdown bounty collected,Shutdown bounty lost,Win,Game_ID,Blue_Team,Red_Team,Winning_Team
0,Adam,TOP,15.0,3,3,3,2,232,4.0,NaN,...,118.0,11.0,28.0,100.0,200.0,0,53697,Team BDS,G2 Esports,G2 Esports
1,Sheo,JUNGLE,14.0,2,2,2,2,213,197.0,NaN,...,46.0,9.0,25.0,0.0,150.0,0,53697,Team BDS,G2 Esports,G2 Esports
2,nuc,MID,14.0,2,3,1,1,224,4.0,NaN,...,90.0,7.0,25.0,0.0,150.0,0,53697,Team BDS,G2 Esports,G2 Esports
3,Ice,ADC,15.0,2,2,2,2,292,7.0,NaN,...,20.0,2.0,17.0,50.0,150.0,0,53697,Team BDS,G2 Esports,G2 Esports
4,Labrov,SUPPORT,11.0,1,4,3,1,44,0.0,NaN,...,95.0,12.0,25.0,0.0,0.0,0,53697,Team BDS,G2 Esports,G2 Esports


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64130 entries, 0 to 64129
Data columns (total 61 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Player                              64130 non-null  object 
 1   Role                                64130 non-null  object 
 2   Level                               59940 non-null  float64
 3   Kills                               64130 non-null  int64  
 4   Deaths                              64130 non-null  int64  
 5   Assists                             64130 non-null  int64  
 6   KDA                                 64130 non-null  object 
 7   CS                                  64130 non-null  int64  
 8   CS in Team's Jungle                 59940 non-null  float64
 9   CS in Enemy Jungle                  0 non-null      float64
 10  CSM                                 64130 non-null  float64
 11  Golds                               64130

In [18]:
def assign_bo5(df):
    # Step 1: Extract unique games
    game_df = df.drop_duplicates(subset='Game_ID').sort_values(by='Game_ID').reset_index(drop=True)

    bo5_flags = [False] * len(game_df)
    series_start = 0

    for i in range(1, len(game_df)):
        teams_now = {game_df.loc[i, 'Blue_Team'], game_df.loc[i, 'Red_Team']}
        teams_prev = {game_df.loc[i - 1, 'Blue_Team'], game_df.loc[i - 1, 'Red_Team']}

        if teams_now == teams_prev:
            continue
        else:
            series_len = i - series_start
            if series_len == 3:
                if Winning_Team := game_df.loc[series_start, 'Winning_Team']:
                    for j in range(series_start, i):
                        if game_df.loc[j, 'Winning_Team'] == Winning_Team:
                            bo5_flags[j] = True
            elif series_len >= 4:
                for j in range(series_start, i):
                    bo5_flags[j] = True
            series_start = i

    game_df['bo5'] = bo5_flags

    # Step 2: Merge bo5 labels back into player-level data
    df = df.merge(game_df[['Game_ID', 'bo5']], on='Game_ID', how='left')
    return df
df = assign_bo5(df)

In [19]:
df.head()

,Player,Role,Level,Kills,Deaths,Assists,KDA,CS,CS in Team's Jungle,CS in Enemy Jungle,...,Consumables purchased,Items Purchased,Shutdown bounty collected,Shutdown bounty lost,Win,Game_ID,Blue_Team,Red_Team,Winning_Team,bo5
0,Adam,TOP,15.0,3,3,3,2,232,4.0,NaN,...,11.0,28.0,100.0,200.0,0,53697,Team BDS,G2 Esports,G2 Esports,False
1,Sheo,JUNGLE,14.0,2,2,2,2,213,197.0,NaN,...,9.0,25.0,0.0,150.0,0,53697,Team BDS,G2 Esports,G2 Esports,False
2,nuc,MID,14.0,2,3,1,1,224,4.0,NaN,...,7.0,25.0,0.0,150.0,0,53697,Team BDS,G2 Esports,G2 Esports,False
3,Ice,ADC,15.0,2,2,2,2,292,7.0,NaN,...,2.0,17.0,50.0,150.0,0,53697,Team BDS,G2 Esports,G2 Esports,False
4,Labrov,SUPPORT,11.0,1,4,3,1,44,0.0,NaN,...,12.0,25.0,0.0,0.0,0,53697,Team BDS,G2 Esports,G2 Esports,False


In [20]:
df.bo5.value_counts()

bo5
False    45510
True     18620
Name: count, dtype: int64